In [146]:
import pandas as pd
import numpy as np
import datetime
import time
from calendar import monthrange

In [86]:
df = pd.read_csv("../../../Data/vw_Incident.csv",encoding='latin-1',low_memory=False)

In [87]:
from sklearn.utils import resample
df = resample(df, n_samples=10, random_state=1234)

In [350]:
# custom month end
def custom_month_end(date):
    
    wkday, days_in_month = monthrange(date.year, date.month)
    lastBDay = days_in_month - max(((wkday + days_in_month - 1) % 7) - 4, 0)
    m=0
    d=31
    increment_day = False
    if date.day == days_in_month and date.day == lastBDay:
#         print("lastBDay is last day in month")
        m=1
        d=1
        increment_day = False
    elif date.day > lastBDay + 1:
#         print("date is a sunday two days after the last business day")
        m=1
        d=31
        increment_day = True
    elif date.day > lastBDay and date.hour >= 8:
#         print("date day after the last business day and outside cutoff")
        m=1
        d=31
        increment_day = True
    elif date.day == 1 and date.hour < 8:
#         print("date is the first day of the month and within cutoff")
        d=1
        increment_day = False
#     elif date.day == 1 and date.hour >= 8:
#         print("date is the first day of the month and outside cutoff")
#         m=1
#         d=31
    elif lastBDay == days_in_month:
#         print("date is during the month and the last business day is the last day of the month")
        increment_day = False
        m=1
        d=1
        
#     elif lastBDay < days_in_month:
        
#     else: print("else")
    date += pd.DateOffset(months=m, day=d)
    
    if date.weekday() > 4:
#         print("weekend, therefore go back to the last business day and add 1 day")
        date-=pd.offsets.BDay()
        date+=pd.offsets.Day()

    elif increment_day == True:
#         print("increment day")
        date+=pd.offsets.Day()
    
        
#     print(date.normalize() + pd.DateOffset(hours=8))
    return date.normalize() + pd.DateOffset(hours=8)
dates = [pd.datetime(2017, 4, 28, 7), pd.datetime(2017, 4, 29, 7), pd.datetime(2017, 4, 30, 12), pd.datetime(2017, 6, 1, 7),
         pd.datetime(2017, 6, 1, 8), pd.datetime(2017, 7, 5, 8), pd.datetime(2017, 7, 31, 8), pd.datetime(2017, 8, 1, 7),
         pd.datetime(2017, 8, 1, 8), pd.datetime(2017, 8, 5, 8), pd.datetime(2017, 8, 20, 16), pd.datetime(2017, 8, 31, 16),
         pd.datetime(2017, 9, 1, 3), pd.datetime(2017, 9, 1, 12), pd.datetime(2017, 9, 10, 12), pd.datetime(2017, 11, 6, 12),
         pd.datetime(2017, 11, 6, 12), pd.datetime(2017, 12, 1, 2), pd.datetime(2017, 12, 1, 9),pd.datetime(2017, 9, 30, 7),
         pd.datetime(2017, 9, 30, 8)
        ]
should_bes = [pd.datetime(2017, 4, 29, 8), pd.datetime(2017, 4, 29, 8), pd.datetime(2017, 6, 1, 8), pd.datetime(2017, 6, 1, 8),
         pd.datetime(2017, 7, 1, 8), pd.datetime(2017, 8, 1, 8), pd.datetime(2017, 8, 1, 8), pd.datetime(2017, 8, 1, 8),
         pd.datetime(2017, 9, 1, 8), pd.datetime(2017, 9, 1, 8), pd.datetime(2017, 9, 1, 8), pd.datetime(2017, 9, 1, 8), 
         pd.datetime(2017, 9, 1, 8), pd.datetime(2017, 9, 30, 8), pd.datetime(2017, 9, 30, 8),pd.datetime(2017, 12, 1, 8),
         pd.datetime(2017, 12, 1, 8), pd.datetime(2017, 12, 1, 8), pd.datetime(2017, 12, 30, 8),pd.datetime(2017, 9, 30, 8),
         pd.datetime(2017, 11, 1, 8)
              
             ]
for date, should_be in zip(dates, should_bes):
    print()
    custom_month_end(date)
    print(should_be)
    
    print()


2017-04-29 08:00:00


2017-04-29 08:00:00


2017-06-01 08:00:00


2017-06-01 08:00:00


2017-07-01 08:00:00


2017-08-01 08:00:00


2017-08-01 08:00:00


2017-08-01 08:00:00


2017-09-01 08:00:00


2017-09-01 08:00:00


2017-09-01 08:00:00


2017-09-01 08:00:00


2017-09-01 08:00:00


2017-09-30 08:00:00


2017-09-30 08:00:00


2017-12-01 08:00:00


2017-12-01 08:00:00


2017-12-01 08:00:00


2017-12-30 08:00:00


2017-09-30 08:00:00


2017-11-01 08:00:00



In [353]:
def seconds_left_in_month(date):    
    cutoff = custom_month_end(date)
#     print(date)
#     print(cutoff)
#     print(cutoff-date)
    seconds_left = (cutoff-date).seconds
    days_left = (cutoff-date).days
#     print(days_left, seconds_left)
    total_seconds_left = seconds_left + days_left*24*60*60
    return total_seconds_left

date = pd.datetime(2017, 4, 29, 7)
seconds_left_in_month(date)

3600

In [355]:
def days_left_in_quarter(date):
    # Function found on stack overflow
    # https://stackoverflow.com/questions/37471704/how-do-i-get-the-correspondent-day-of-the-quarter-from-a-date-field
    q2 = (datetime.datetime.strptime("4/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    print(q2)
    q3 = (datetime.datetime.strptime("7/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q4 = (datetime.datetime.strptime("10/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q5 = (datetime.datetime.strptime("12/31/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday + 1 # 1st Jan

    cur_day =  date.timetuple().tm_yday
    print(cur_day)
    if (date.month < 4):
        return q2 - (cur_day)
    elif (date.month < 7):
        return q3 - (cur_day - q2 + 1)
    elif (date.month < 10):
        return q4 - (cur_day - q3 + 1)
    else:
        return q5 - (cur_day - q4 + 1)
def days_left_in_month(date):
    return int(monthrange(date.year, date.month)[1]) - int(date.strftime("%d"))

def time_taken(df, out_file, start, finish, d):  # replace start & finish with one new column, "TimeTaken"
    df[start] = pd.to_datetime(df[start])
    df[finish] = pd.to_datetime(df[finish])
    df2 = pd.DataFrame()  # create new dataframe, df2, to store answer  # todo - no need for df2
    df2["TimeTaken"] = (df[finish] - df[start]).astype('timedelta64[s]')
    # del df[start]  # Removed so we can include time to month and qtr end
#     if d["delete_created_resolved"] == "y":
#         del df[finish]
    df = pd.concat([df2, df], axis=1)
#     out_file.write("\nTime Taken column calculated" + "\n")
#     mean_time = sum(df["TimeTaken"].tolist()) / len(df["TimeTaken"])  # Calculate mean of time taken
#     std_time = np.std(df["TimeTaken"].tolist())  # Calculate standard deviation of time taken
#     df = df[df["TimeTaken"] < (mean_time + 3*std_time)]  # Remove outliers that are > 3 std from mean
###     df = df[df["TimeTaken"] < 2000000]  # Remove outliers that are > 2000000
#     out_file.write("Outliers removed > 3 sd from mean of TimeTaken" + "\n")
    df["Days_left_Month"] = df["Created_On"].apply(lambda x: int(days_left_in_month(x)))  # Day of the month
    df["Seconds_left_month"] = df["Created_On"].apply(lambda x: int(seconds_left_in_month(x)))  # Day of the month
#     out_file.write("Day of month calculated" + "\n")
    df["Days_left_QTR"] = df["Created_On"].apply(lambda x: int(days_left_in_quarter(x)))  # Day of the Qtr
#     if d["delete_created_resolved"] == "y":
#         del df["Created_On"]
#     out_file.write("Day of quarter calculated" + "\n\n")
    return df
out_file=None
d=None
df = time_taken(df, out_file, "Created_On", "ResolvedDate", d)  # Create Time Variable and filter outliers
df.head()

91
151
91
66
91
100
91
110
91
66
91
103
91
95
91
87
91
66
91
65


,TimeTaken,TimeTaken,TimeTaken,TicketNumber,IncidentId,Created_On,Receiveddate,ResolvedDate,Queue,StatusReason,...,Numberofreactivations,NumberofChildIncidents,ParentCase,Referencesystem,StateCode,Isrevenueimpacting,IsSOXCase,Days_left_Month,Seconds_left_month,Days_left_QTR
27439,24675.0,24675.0,24675.0,5-0000000820000,B70261FE-3E46-E711-80F7-3863BB3C66F8,2017-05-31 20:23:31,2017-05-31 20:23:31,2017-06-01 03:14:46,<VL NAOC Orders>,Problem Solved,...,NaN,NaN,NaN,NaN,Resolved,0,1.0,0,41789,121
34086,266460.0,266460.0,266460.0,5-0000000443302,08DF3732-3403-E711-80F8-FC15B4289B54,2017-03-07 12:47:25,2017-03-07 12:47:23,2017-03-10 14:48:25,<VL EOC Enterprise Orders and Adjustments>,Completed,...,NaN,NaN,NaN,NaN,Resolved,0,NaN,24,2142755,25
23924,1889729.0,1889729.0,1889729.0,5-0000000600773,3EBCA871-1E1E-E711-80EE-3863BB3C66F8,2017-04-10 18:49:42,2017-04-10 18:49:42,2017-05-02 15:45:11,<VL AOC Outbound Shipping>,Problem Solved,...,NaN,NaN,NaN,NaN,Resolved,0,1.0,20,1602618,172
17048,161736.0,161736.0,161736.0,5-0000000637441,B8AF68D7-6C25-E711-80F1-3863BB3C0158,2017-04-20 01:58:28,2017-04-20 02:00:00,2017-04-21 22:54:04,<VL NAOC Order Rework>,Problem Solved,...,NaN,NaN,NaN,NaN,Resolved,0,1.0,10,799292,162
32399,84527.0,84527.0,84527.0,5-0000000442502,575EE584-2103-E711-80F3-FC15B4284D68,2017-03-07 10:33:45,2017-03-07 10:33:29,2017-03-08 10:02:32,<VL APOC Enterprise>,Rejected,...,NaN,NaN,NaN,NaN,Resolved,0,NaN,24,2150775,25


In [130]:
def seconds_left_in_month(date):    
    cutoff = (date + pd.offsets.BMonthEnd(0)).normalize() + pd.DateOffset(hours=32)
    print(date)
    print(cutoff)
    seconds_left = (cutoff-date).seconds
    days_left = (cutoff-date).days
    print(days_left, seconds_left)
    total_seconds_left = seconds_left + days_left*24*60*60
    return total_seconds_left

date = pd.datetime(2017, 4, 29, 7)
seconds_left_in_month(date)

2017-04-29 07:00:00
2017-06-01 08:00:00
33 3600


2854800

In [161]:
class BusinessMonthEnd(MonthOffset):
    """DateOffset increments between business EOM dates"""

    def isAnchored(self):
        return (self.n == 1)

    @apply_wraps
    def apply(self, other):
        n = self.n
        wkday, days_in_month = tslib.monthrange(other.year, other.month)
        lastBDay = days_in_month - max(((wkday + days_in_month - 1)
                                        % 7) - 4, 0)

        if n > 0 and not other.day >= lastBDay:
            n = n - 1
        elif n <= 0 and other.day > lastBDay:
            n = n + 1
        other = other + relativedelta(months=n, day=31)

        if other.weekday() > 4:
            other = other - pd.BDay()
        return other

NameError: name 'MonthOffset' is not defined

In [167]:
# first_created_on = df["Created_On"].min()
# last_created_on  = df["Created_On"].max()
last_bdays = pd.date_range("2017.01.01", periods=11, freq='BM')
last_bdays_offset = []
for last_bday in last_bdays:
    last_bdays_offset.append(last_bday + pd.DateOffset(days=1,hours=8))
print(last_bdays_offset)

def seconds_left_in_month(date, last_bdays_offset):
    print(date.month)
    cutoff = (date + pd.offsets.BMonthEnd(0)).normalize()
    print(date)
    print(cutoff)
    

date = pd.datetime(2017, 4, 30, 7)
seconds_left_in_month(date,last_bdays_offset)

[Timestamp('2017-02-01 08:00:00'), Timestamp('2017-03-01 08:00:00'), Timestamp('2017-04-01 08:00:00'), Timestamp('2017-04-29 08:00:00'), Timestamp('2017-06-01 08:00:00'), Timestamp('2017-07-01 08:00:00'), Timestamp('2017-08-01 08:00:00'), Timestamp('2017-09-01 08:00:00'), Timestamp('2017-09-30 08:00:00'), Timestamp('2017-11-01 08:00:00'), Timestamp('2017-12-01 08:00:00')]
4
2017-04-30 07:00:00
2017-05-31 00:00:00


In [22]:
start = pd.datetime(2017, 2, 1)
end = pd.datetime(2017, 7, 1)
print((start + pd.offsets.BMonthEnd(1)).day)
# df['BMonthEnd'] = (df.index + pd.offsets.BMonthEnd(1)).day
end_of_months = pd.date_range(start, end, freq='BM')

last_bdayslist = []
for end_of_month in end_of_months:
    last_bdayslist += pd.bdate_range(end_of_month- BDay(3),end_of_month- BDay(0))

dflast_bdays = pd.DataFrame(last_bdayslist, columns=["Last_4_BDays"])
dflast_bdays["Last_4_BDays"]

28


NameError: name 'BDay' is not defined

In [18]:
def days_left_in_quarter(date):
    # Function found on stack overflow
    # https://stackoverflow.com/questions/37471704/how-do-i-get-the-correspondent-day-of-the-quarter-from-a-date-field
    q2 = (datetime.datetime.strptime("4/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q3 = (datetime.datetime.strptime("7/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q4 = (datetime.datetime.strptime("10/1/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday
    q5 = (datetime.datetime.strptime("12/31/{0:4d}".format(date.year), "%m/%d/%Y")).timetuple().tm_yday + 1 # 1st Jan

    cur_day =  date.timetuple().tm_yday
    if (date.month < 4):
        return q2 - (cur_day)
    elif (date.month < 7):
        return q3 - (cur_day - q2 + 1)
    elif (date.month < 10):
        return q4 - (cur_day - q3 + 1)
    else:
        return q5 - (cur_day - q4 + 1)
def days_left_in_month(date):
    return int(monthrange(date.year, date.month)[1]) - int(date.strftime("%d"))

def time_taken(df, out_file, start, finish, d):  # replace start & finish with one new column, "TimeTaken"
    df[start] = pd.to_datetime(df[start])
    df[finish] = pd.to_datetime(df[finish])
    df2 = pd.DataFrame()  # create new dataframe, df2, to store answer  # todo - no need for df2
    df2["TimeTaken"] = (df[finish] - df[start]).astype('timedelta64[s]')
    # del df[start]  # Removed so we can include time to month and qtr end
#     if d["delete_created_resolved"] == "y":
#         del df[finish]
    df = pd.concat([df2, df], axis=1)
#     out_file.write("\nTime Taken column calculated" + "\n")
#     mean_time = sum(df["TimeTaken"].tolist()) / len(df["TimeTaken"])  # Calculate mean of time taken
#     std_time = np.std(df["TimeTaken"].tolist())  # Calculate standard deviation of time taken
#     df = df[df["TimeTaken"] < (mean_time + 3*std_time)]  # Remove outliers that are > 3 std from mean
###     df = df[df["TimeTaken"] < 2000000]  # Remove outliers that are > 2000000
#     out_file.write("Outliers removed > 3 sd from mean of TimeTaken" + "\n")
    df["Days_left_Month"] = df["Created_On"].apply(lambda x: int(days_left_in_month(x)))  # Day of the month
#     out_file.write("Day of month calculated" + "\n")
    df["Days_left_QTR"] = df["Created_On"].apply(lambda x: int(days_left_in_quarter(x)))  # Day of the Qtr
#     if d["delete_created_resolved"] == "y":
#         del df["Created_On"]
#     out_file.write("Day of quarter calculated" + "\n\n")
    return df
out_file=None
d=None
df = time_taken(df, out_file, "Created_On", "ResolvedDate", d)  # Create Time Variable and filter outliers
df.head()

,TimeTaken,TimeTaken,TicketNumber,IncidentId,Created_On,Receiveddate,ResolvedDate,Queue,StatusReason,Priority,...,Complexity,Numberofreactivations,NumberofChildIncidents,ParentCase,Referencesystem,StateCode,Isrevenueimpacting,IsSOXCase,Days_left_Month,Days_left_QTR
27439,24675.0,24675.0,5-0000000820000,B70261FE-3E46-E711-80F7-3863BB3C66F8,2017-05-31 20:23:31,2017-05-31 20:23:31,2017-06-01 03:14:46,<VL NAOC Orders>,Problem Solved,Normal,...,Medium,NaN,NaN,NaN,NaN,Resolved,0,1.0,0,121
34086,266460.0,266460.0,5-0000000443302,08DF3732-3403-E711-80F8-FC15B4289B54,2017-03-07 12:47:25,2017-03-07 12:47:23,2017-03-10 14:48:25,<VL EOC Enterprise Orders and Adjustments>,Completed,Normal,...,Medium,NaN,NaN,NaN,NaN,Resolved,0,NaN,24,25
23924,1889729.0,1889729.0,5-0000000600773,3EBCA871-1E1E-E711-80EE-3863BB3C66F8,2017-04-10 18:49:42,2017-04-10 18:49:42,2017-05-02 15:45:11,<VL AOC Outbound Shipping>,Problem Solved,Immediate,...,NaN,NaN,NaN,NaN,NaN,Resolved,0,1.0,20,172
17048,161736.0,161736.0,5-0000000637441,B8AF68D7-6C25-E711-80F1-3863BB3C0158,2017-04-20 01:58:28,2017-04-20 02:00:00,2017-04-21 22:54:04,<VL NAOC Order Rework>,Problem Solved,Normal,...,Medium,NaN,NaN,NaN,NaN,Resolved,0,1.0,10,162
32399,84527.0,84527.0,5-0000000442502,575EE584-2103-E711-80F3-FC15B4284D68,2017-03-07 10:33:45,2017-03-07 10:33:29,2017-03-08 10:02:32,<VL APOC Enterprise>,Rejected,Normal,...,Medium,NaN,NaN,NaN,NaN,Resolved,0,NaN,24,25
